# Test Colab Bootstrap and Badge
This demos some code to facilitate running notebooks in Colab rather than a conda environment

In [1]:
# --- One first cell for VS Code, JupyterLab Desktop, and Colab ---
import os, sys, subprocess, importlib, pathlib, re

# 0) Notebook path (keeps badge correct) — relative to repo root
os.environ["NB_PATH"] = "demos/TestingDemos/TestColabBootstrapAndBadge.ipynb"
os.environ["NOTEBOOK_HEADER_AUTORUN"] = "0"  # we call main() explicitly

# Helper: parse org/repo from a git remote URL
def _parse_org_repo(url: str):
    m = re.search(r"[:/](?P<org>[^/]+)/(?P<repo>[^/\\.]+)(?:\\.git)?$", url or "")
    return (m.group("org"), m.group("repo")) if m else (None, None)

# 1) Try to detect org/repo from local git (works in VS Code + JupyterLab Desktop)
ORG = os.environ.get("BOOT_ORG")
REPO = os.environ.get("BOOT_REPO")
if not (ORG and REPO):
    try:
        remote = subprocess.check_output(
            ["git", "config", "--get", "remote.origin.url"], text=True
        ).strip()
        ORG, REPO = _parse_org_repo(remote)
    except Exception:
        ORG = REPO = None

# 2) If we're in Colab and still don't know org/repo, try to infer; else fall back
IN_COLAB = ("COLAB_RELEASE_TAG" in os.environ) or ("COLAB_GPU" in os.environ)
if IN_COLAB and not (ORG and REPO):
    # Try a few heuristics; if none work, default to QMCSoftware/QMCSoftware.
    # You can always override by setting BOOT_ORG/BOOT_REPO above.
    # Heuristic A: sometimes Colab exposes the notebook URL
    ORG = ORG or os.environ.get("COLAB_GITHUB_ORG")
    REPO = REPO or os.environ.get("COLAB_GITHUB_REPO")
    if not (ORG and REPO):
        ORG, REPO = "QMCSoftware", "QMCSoftware"

# 3) Colab: clone if needed; Local: use repo_root
if IN_COLAB:
    repo_dir = f"/content/{REPO}"
    if not pathlib.Path(repo_dir).exists():
        branch = os.environ.get("BOOT_BRANCH", "bootstrap_colab")
        subprocess.check_call([
            "git", "clone", "--depth", "1", "--recurse-submodules",
            "-b", branch, f"https://github.com/{ORG}/{REPO}.git", repo_dir
        ])
    sys.path.insert(0, os.path.join(repo_dir, "utils"))
    try: os.chdir(repo_dir)
    except Exception: pass
else:
    repo_root = subprocess.check_output(
        ["git", "rev-parse", "--show-toplevel"], text=True
    ).strip()
    sys.path.insert(0, os.path.join(repo_root, "utils"))

# 4) Import the header and run (quiet by default)
import notebook_header
importlib.reload(notebook_header)
notebook_header.main(force=True)  # quiet=True is the default

In [2]:
# Optional: turn LaTeX on before running the header in Colab
# %env USE_TEX=1
# (Re-run the first cell afterwards to apply it)

In [3]:
# ---- Sanity check: matplotlib mode ----
import matplotlib as mpl
print("usetex?", mpl.rcParams.get("text.usetex"))

# ---- Sanity check: qmcpy ----
import qmcpy as qp
print(qp.Sobol(3).gen_samples(8))

usetex? False
[[0.20274731 0.75603777 0.42146013]
 [0.87379103 0.46270154 0.93759324]
 [0.43722866 0.23909574 0.74654417]
 [0.51455379 0.54219505 0.14452119]
 [0.049777   0.33732887 0.03952367]
 [0.87856798 0.88139367 0.57026546]
 [0.31529473 0.66757199 0.8707191 ]
 [0.73780609 0.1137364  0.26961719]]
